<a href="https://colab.research.google.com/github/mainaroseline/Mobile-Phone-Sentiment-Analysis/blob/main/Mobile_Phone_Sentiment_Analysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 1. Business Understanding

## 1.1. Problem Statement

Purchasing a product is an interaction between two entities, consumers and business owners. Consumers often use reviews to make decisions about what products to buy, while businesses, on the other hand, not only want to sell their products but also want to receive feedback in terms of consumer reviews. Consumer reviews about purchased products shared on the internet have a great impact. Human nature is generally structured to make decisions based on analyzing and getting the benefit of other consumer experience and opinions because others often have a great influence on our beliefs, behaviors, perception of reality, and the choices we make. Hence, we ask others for their feedback whenever we are deciding on doing something. Additionally, this fact applies not only to consumers but also to organizations and institutions.

As social media networks have evolved, so have the ways that consumers express their opinions and feelings. With the vast amount of data now available online, it has become a challenge to extract useful information from it all. Sentiment analysis has emerged as a way to predict the polarity (positive, negative, or neutral) of consumer opinion, which can help consumers better understand the textual data.

E-commerce websites have increased in popularity to the point where consumers rely on them for buying and selling. These websites give consumers the ability to write comments about different products and services, which has resulted in a huge amount of reviews becoming available. Consequently, the need to analyze these reviews to understand consumers’ feedback has increased for both vendors and consumers. However, it is difficult to read all the feedback for a particular item, especially for popular items with many comments. 

In this research, we attempt to build a predictor for consumers’ satisfaction on mobile phone products based on the reviews. We will also attempt to understand the factors that contribute to classifying reviews as positive, negative or neutral (based on important or most frequent words). This is believed to help companies improve their products and also help potential buyers make better decisions when buying products.


### 1.1.1. Main objective

- To perform a sentiment analysis of mobile phone reviews from Amazon website to determine how these reviews help consumers to have conﬁdence that they have made the right decision about their purchases.

### 1.1.2. Specific Objectives

- To help companies understand their consumers’ feedback to maintain their products/services or enhance them.
- To provide insights to companies in curating offers on speciﬁc products to increase their proﬁts and customer satisfaction.
- To understand the factors that contribute to classifying reviews as positive, negative or neutral (based on important or most frequent words).
- To determine mobile phones key features that influence smartphone purchases.
- To perform a market segmentation of consumers based on their reviews
- To advise the advertisement department in companies on these key features to use as selling points and to specific customer segments  in upcoming advertisements.

### 1.1.3. Metrics of Success

The best performing model will be selected based on:
- An accuracy score > 80%
- An F1 score > 0.85 


# 2. Data Understanding

The data used for this project is obtained from [data.world](https://data.world/promptcloud/amazon-mobile-phone-reviews) and contains more than 400 thousand reviews  of unlocked mobile phones sold on [amazon.com](https://www.amazon.com/). The data was collected from 2016 and last updated in April 2022. The data contains 6 columns:
- Product_name : Contains the name of the product
- Brand : Contains the brand of the product
- Price : Contains the price of the brans
- Rating : Contains the rating awarded to that product
- Reviews : Contains the review of that product
- Review_votes : Number of people who found the review helpful



# 3. Loading Relevant Libraries & Data

## 3.1. Loading Libraries

In [1]:
import pandas as pd

import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
import seaborn as sns
from wordcloud import WordCloud, STOPWORDS


import nltk
import string
import re
from nltk import pos_tag
from nltk.probability import FreqDist
from nltk.stem import WordNetLemmatizer
from nltk.corpus import stopwords, wordnet
from nltk.tokenize import RegexpTokenizer, word_tokenize


from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer


from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report, confusion_matrix

!pip install contractions
import contractions

# nltk downloads
nltk.download('wordnet', quiet=True)
nltk.download('punkt', quiet=True)
nltk.download('stopwords', quiet=True)
nltk.download('tagsets', quiet=True)
nltk.download('averaged_perceptron_tagger', quiet=True)

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 287 kB 8.1 MB/s 
     |████████████████████████████████| 106 kB 72.8 MB/s 


True

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


## 3.2. Loading Data

In [3]:
# loading the data

df = pd.read_csv('/content/drive/Shareddrives/Alpha/Data/Amazon Combined Data.csv')
df.head()

,Rating,Review Title,Review,Location and Date of Review,Affiliated Company,Brand and Features,Price
0,4.0 out of 5 stars,"\n.. not what ordered, not New... but it works...","\nSo first off...it's not what I ordered, but ...","Reviewed in the United States on February 11, ...",Visit the RCA Store,"RCA Reno Smartphone, 4G LTE, 16GB, And...",$69.99
1,3.0 out of 5 stars,\nNot for Cricket Wireless and this two review...,"\nThe phone itself is a okay android device, b...","Reviewed in the United States on February 4, 2021",Visit the RCA Store,"RCA Reno Smartphone, 4G LTE, 16GB, And...",$69.99
2,3.0 out of 5 stars,\nWill not work on T-Mobile sysem!\n,\nNew phone write up indicates T-Mobile system...,"Reviewed in the United States on June 7, 2022",Visit the RCA Store,"RCA Reno Smartphone, 4G LTE, 16GB, And...",$69.99
3,3.0 out of 5 stars,\nA burner or for a kid\n,\nI use this as a burner w/o a sim card in it....,"Reviewed in the United States on April 14, 2022",Visit the RCA Store,"RCA Reno Smartphone, 4G LTE, 16GB, And...",$69.99
4,4.0 out of 5 stars,\nIt works okay\n,\nIt works fine\n,"Reviewed in the United States on August 13, 2022",Visit the RCA Store,"RCA Reno Smartphone, 4G LTE, 16GB, And...",$69.99


## 3.3. Previewing Data

In [4]:
# checking the shape of the data

print(f'The data has {df.shape[0]} rows and {df.shape[1]} columns')

The data has 17198 rows and 7 columns


In [5]:

# checking the data types of the data

df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 17198 entries, 0 to 17197
Data columns (total 7 columns):
 #   Column                       Non-Null Count  Dtype 
---  ------                       --------------  ----- 
 0   Rating                       17198 non-null  object
 1   Review Title                 17198 non-null  object
 2   Review                       17168 non-null  object
 3   Location and Date of Review  17198 non-null  object
 4   Affiliated Company           17198 non-null  object
 5   Brand and Features           17198 non-null  object
 6   Price                        17198 non-null  object
dtypes: object(7)
memory usage: 940.6+ KB


# 4. Data Cleaning

## 4.1. Missing values
- Checking for missing values


In [6]:
# Define a function to get missing data
def missing_data(data):
    missing_data = data.isna().sum()
    missing_data = missing_data[missing_data>0]
    return missing_data

In [7]:
# Getting the sum of missing values per column

missing_data(df).to_frame()

,0
Review,30


Out of the 7 columns, only the review's column has missing values.

Since the dataset is large, the missing values can be dropped and still retain relevant information.

In [8]:
# Dropping the missing values

df.dropna(inplace = True)

# Confirming there are no missing values 

missing_data(df).to_frame()

,0


## 4.2. Duplicates

In [9]:
# Checking for duplicates

print(f"The data has {df.duplicated().sum()} duplicated rows")

The data has 6595 duplicated rows


In [10]:
# Exploring the duplicates

duplicates = df[df.duplicated(keep = 'first')]

duplicates.head(4)

,Rating,Review Title,Review,Location and Date of Review,Affiliated Company,Brand and Features,Price
30,4.0 out of 5 stars,"\n.. not what ordered, not New... but it works...","\nSo first off...it's not what I ordered, but ...","Reviewed in the United States on February 11, ...",Visit the RCA Store,"RCA Reno Smartphone, 4G LTE, 16GB, And...",$69.99
31,3.0 out of 5 stars,\nNot for Cricket Wireless and this two review...,"\nThe phone itself is a okay android device, b...","Reviewed in the United States on February 4, 2021",Visit the RCA Store,"RCA Reno Smartphone, 4G LTE, 16GB, And...",$69.99
32,3.0 out of 5 stars,\nWill not work on T-Mobile sysem!\n,\nNew phone write up indicates T-Mobile system...,"Reviewed in the United States on June 7, 2022",Visit the RCA Store,"RCA Reno Smartphone, 4G LTE, 16GB, And...",$69.99
33,3.0 out of 5 stars,\nA burner or for a kid\n,\nI use this as a burner w/o a sim card in it....,"Reviewed in the United States on April 14, 2022",Visit the RCA Store,"RCA Reno Smartphone, 4G LTE, 16GB, And...",$69.99


Duplicated columns will be dropped to avoid it misguiding our analysis and prediction process.

In [11]:
# Dropping the duplicates

df.drop_duplicates(inplace = True)


In [12]:
# Confirming if there are duplicates

df.duplicated().sum()

0

## 4.3. Cleaning Specific Columns

### 4.3.1. Rating Column

For better analysis the rating value needs to be extracted and in a float/integer format.

In [13]:
#Extracting the digits in the Rating column and converting it to string type

df["Rating"] = df["Rating"].str.extract('(\d+)').astype(str)
df["Rating"].head().to_frame()

,Rating
0,4
1,3
2,3
3,3
4,4



The rating value was extracted from the column and converted into float data type

### 4.3.2. Price Column

The price columns seems has a dollar sign which could affect analysis. It needs stripping and conversion to int/float

In [14]:
#Extracting the digits in the price column and converting it to integer

df["Price"] = df["Price"].str.extract('(\d+)').astype(int)

df["Price"].head().to_frame()

,Price
0,69
1,69
2,69
3,69
4,69


The dollar sign was removed and price column converted into integer data type

### 4.3.3. Affiliated company column

In [15]:
# Rename the column to brand name

df.rename(columns = {"Affiliated Company":"Brand","Brand and Features":"Product_name"},inplace = True)
df.head(3)

,Rating,Review Title,Review,Location and Date of Review,Brand,Product_name,Price
0,4,"\n.. not what ordered, not New... but it works...","\nSo first off...it's not what I ordered, but ...","Reviewed in the United States on February 11, ...",Visit the RCA Store,"RCA Reno Smartphone, 4G LTE, 16GB, And...",69
1,3,\nNot for Cricket Wireless and this two review...,"\nThe phone itself is a okay android device, b...","Reviewed in the United States on February 4, 2021",Visit the RCA Store,"RCA Reno Smartphone, 4G LTE, 16GB, And...",69
2,3,\nWill not work on T-Mobile sysem!\n,\nNew phone write up indicates T-Mobile system...,"Reviewed in the United States on June 7, 2022",Visit the RCA Store,"RCA Reno Smartphone, 4G LTE, 16GB, And...",69


The columns affiliated company and brand&features was renamed to brand and product name respectively.

In [16]:
#Getting the value counts for the brand column

df['Brand'].value_counts().to_frame()

,Brand
Visit the Amazon Renewed Store,1747
Brand: Motorola,1578
Visit the BLU Store,1544
Visit the TCL Store,1448
Brand: Amazon Renewed,989
Visit the OnePlus Store,684
Visit the SAMSUNG Store,522
Visit the Nokia Store,492
Visit the Google Store,389
Visit the JTEMAN Store,263


- The brand name will be extracted.
- white space will be stripped.
- refurbished phones will be renamed into one name

In [17]:
# Removing unnecessary words from the column to get the brand name
word_vocabulary = ['Visit', 'the', 'store', 'Brand:', 'Store']
for word in word_vocabulary:
    df['Brand'] = df['Brand'].str.replace(word, '')

# Removing all the white spaces
df['Brand'] = df['Brand'].str.strip()

# Renaming the amazon renewed with refurbished
df['Brand'] = df['Brand'].str.replace('Amazon Renewed','Amazon Refurbished')  

# 5. Feature Engineering

### 5.1 Product_name column

product name column model name in it. It will be stripped into a new column. 
Some of the model names have the word smartphone that will be stripped as well.

In [18]:
#removing punctuations from the column

df['Product_name'] = df['Product_name'].str.replace(r"\(.*\)","", regex=True)
df['Product_name'] = df['Product_name'].str.replace('-',"", regex=True)
df['Product_name'] = df['Product_name'].str.replace(',',"", regex=True)
df['Product_name'] = df['Product_name'].str.replace('|',"", regex=True)

#Splitting the strings in this column into different columns

string_cols = df["Product_name"].str.split(" ", n = -1, expand = True)

#selecting on the first three words of the string that will form the phone type
df["first_word"] = string_cols[8]
df["middle_word"] = string_cols[9]
df["last_word"] = string_cols[10]

#copying the two other columns so as to allow concactination
new1 = df["middle_word"].copy()
new2 = df["last_word"].copy()
 
# concatenating team with name column
# overwriting name column
df["Model_Type"]= df["first_word"].str.cat(new1, sep =" ")
df["Model_Type"]= df["Model_Type"].str.cat(new2, sep =" ")
df.drop(["first_word", "middle_word", "last_word"], axis=1, inplace=True)

# Removing unnecessary words from the model type column
word_vocabulary = ['Smartphone']
for word in word_vocabulary:
    df['Model_Type'] = df['Model_Type'].str.replace(word, '')


### 5.1.1 Location and Date of Review

This data was all collected from the United States. This will will be stripped and the dates for reviews will be left

In [19]:
# Extract the review dates from the Location and Date of Review Column
word_vocabulary = ['Reviewed in the', 'United States', 'on', 'January 1, 2022']
for word in word_vocabulary:
    df['Location and Date of Review'] = df['Location and Date of Review'].str.replace(word, '')


# Rename the column to Review Date
df.rename(columns = {'Location and Date of Review': 'Review Date'}, inplace = True)

# Convert the column into datetime format
df['Review Date'] = pd.to_datetime(df['Review Date'], errors = 'coerce')

### 5.1.2. Review Title and Review 

Review and Review Title have some rows that are not in Egnlish. 
These rows will be dropped.


In [20]:
# Explore review title rows not in English
df['Review Title'].iloc[7:10].to_frame()

,Review Title
7,\nBuena Compra\n
8,\nEso no me gustó\n
9,"\nDemasiado básico y lento, bajo costo pero no..."


In [21]:
# Explore review rows not in English
df['Review'].iloc[7:10].to_frame()

,Review
7,\nTal. Como està descrito….Todo lo necesario a...
8,\nNo vale la pena gastar dinero en el.\n
9,"\nDemasiado básico y lento, bajo costo pero no..."


In [22]:
# Drop rows not in English
df = df[df['Review Title'].map(lambda x: x.isascii())]
df = df[df['Review'].map(lambda x: x.isascii())]

### 5.1.3. Reorder columns in dataframe.
the columns will be reordered to have reviews as the last column

In [23]:
df = df.reindex(columns=['Product_name', 'Model_Type', 'Brand', 'Price', 'Review Date', 
                         'Rating', 'Review Title', 'Review'])

Explore cleaned dataframe

In [24]:
# Explore cleaned dataframe
df.head()

,Product_name,Model_Type,Brand,Price,Review Date,Rating,Review Title,Review
0,RCA Reno Smartphone 4G LTE 16GB Androi...,RCA Reno,RCA,69,2022-02-11,4,"\n.. not what ordered, not New... but it works...","\nSo first off...it's not what I ordered, but ..."
2,RCA Reno Smartphone 4G LTE 16GB Androi...,RCA Reno,RCA,69,2022-06-07,3,\nWill not work on T-Mobile sysem!\n,\nNew phone write up indicates T-Mobile system...
3,RCA Reno Smartphone 4G LTE 16GB Androi...,RCA Reno,RCA,69,2022-04-14,3,\nA burner or for a kid\n,\nI use this as a burner w/o a sim card in it....
4,RCA Reno Smartphone 4G LTE 16GB Androi...,RCA Reno,RCA,69,2022-08-13,4,\nIt works okay\n,\nIt works fine\n
5,RCA Reno Smartphone 4G LTE 16GB Androi...,RCA Reno,RCA,69,2022-05-10,3,\nPhone\n,"\nSo far I don't like this phone at all, I thr..."


In [25]:
df['Review Title'] = df['Review Title'].str.strip()
df['Review'] = df['Review'].str.strip()
df.head()

,Product_name,Model_Type,Brand,Price,Review Date,Rating,Review Title,Review
0,RCA Reno Smartphone 4G LTE 16GB Androi...,RCA Reno,RCA,69,2022-02-11,4,".. not what ordered, not New... but it works s...","So first off...it's not what I ordered, but I ..."
2,RCA Reno Smartphone 4G LTE 16GB Androi...,RCA Reno,RCA,69,2022-06-07,3,Will not work on T-Mobile sysem!,New phone write up indicates T-Mobile system c...
3,RCA Reno Smartphone 4G LTE 16GB Androi...,RCA Reno,RCA,69,2022-04-14,3,A burner or for a kid,I use this as a burner w/o a sim card in it. J...
4,RCA Reno Smartphone 4G LTE 16GB Androi...,RCA Reno,RCA,69,2022-08-13,4,It works okay,It works fine
5,RCA Reno Smartphone 4G LTE 16GB Androi...,RCA Reno,RCA,69,2022-05-10,3,Phone,"So far I don't like this phone at all, I threw..."


In [26]:
# Confirming new changes have created new missing values and if yes drop them
print(missing_data(df))
df.dropna(inplace = True)

Review Date    12
dtype: int64


In [27]:
# Explore the shape of the cleaned dataframe
df.shape

(9144, 8)

In [28]:
# Dropping  unnecessary columns

df.drop('Product_name', axis=1, inplace=True)

# 6. Exploratory Data Analysis(EDA)

- Create a Pandas Profile
- Find out the  relationship between product rating and reviews.
- Explore the  relationship between brand and price.
- Explore  the relationship between brand and number of reviews
- Word cloud of most-used words in reviews.
- Trend of reviews over the years.
- Find out the relationship between price and product rating.




### 6.1 Pandas Profiling


In [29]:
import pandas_profiling

from pandas_profiling import ProfileReport


In [30]:
#report = ProfileReport(df, title='Pandas Profiling Report')
#report

The pandas profile

In [31]:
# report.to_file(output_file='Amazon_Pandas_profile.html')

Summary of Profile Report:

Overview
- The dataset has 9144 rows and 9 columns, 6 of which are categorical, 1 is a date column and 2 are numerical variables.

Variables

- Minimum price is 1 dollar and maximum price is 799 dollars. This might show the presence of outliers that might need to be addressed later.
- Top smartphone brands within the dataset are; Amazon Refurbished, Motorola, Blu, TLC and OnePlus.
- The review dates range from Novemver 2014 to September 2022.

- Most common words in the review column are Good, Love it, excelente, great phone and nice.

- - Most common words in the review title column are great phone, good phone and good.


Missing Values

- No missing values in the dataset.

### 6.2. Relationship between ratings and reviews

In [32]:
# Group by relevant columns
ratings_review = df.groupby('Rating')['Review'].count()

In [33]:
# plot the data
ax1 = ratings_review.plot(kind='bar', figsize=(15,8), color="green", fontsize = 13);
ax1.set_alpha(0.8)
ax1.set_title('Distribution of Reviews by Product Rating', fontsize = 20)
ax1.set_ylabel("Number of Reviews", fontsize = 15);
ax1.set_xlabel("Ratings", fontsize = 15 )
plt.show();


Mobile phones with higher rating receive the highest number of reviews followed by phones with the least review rating.

### 6.3. Brands with the highest reviews and with distribution of average prices

In [34]:
# Group relevant columns
brand_reviews = df.groupby('Brand')['Review'].count().sort_values(ascending = False).head(10)
brand_prices = df.groupby('Brand')['Price'].max().sort_values(ascending = False).head(10)

In [35]:
# Plot the data
ax1 = brand_reviews.plot(kind = 'bar', figsize = (15,8), color = 'green', fontsize = 13);
ax1.set_alpha(0.8)
ax1.set_title('Number of Reviews by Brands', fontsize = 26)
ax1.set_ylabel('Number of Reviews', fontsize = 20);
ax1.set_xlabel('Brand', fontsize = 20)
plt.show();



In [36]:
ax2 = brand_prices.plot(kind = 'bar', figsize = (15,8), color = 'green', fontsize = 13);
ax2.set_alpha(0.8)
ax2.set_title('Priciest Phones by Brands', fontsize = 26)
ax2.set_ylabel('Price', fontsize = 20);
ax2.set_xlabel('Brand', fontsize = 20)
plt.xticks(fontsize = 18)
plt.show();

Refurbished phones have the highest number of reviews while Samsung phones have the priciest phones. Samsung is one of the top phone brands in the world and with high performance, which could explain the high prices of its models.

### 6.4. Word cloud of the most-used words in reviews.

In [37]:
# Plot a word cloud 
comment_words = ''
stopwords = set(STOPWORDS)
 
# iterate through the csv file
for val in df.Review:
     
    # typecaste each val to string
    val = str(val)
 
    # split the value
    tokens = val.split()
     
    # Converts each token into lowercase
    for i in range(len(tokens)):
        tokens[i] = tokens[i].lower()
     
    comment_words += " ".join(tokens)+" "
 
wordcloud = WordCloud(width = 800, height = 800,
                background_color ='white',
                stopwords = stopwords,
                min_font_size = 10).generate(comment_words)
 
# plot the WordCloud image                      
plt.figure(figsize = (8, 8), facecolor = None)
plt.imshow(wordcloud)
plt.axis("off")
plt.tight_layout(pad = 0)
 
plt.show() 

good, great, battery price, issue and work are some of the words that pop up more in the reviews

### 6.5. Trend of reviews over the years

In [38]:
# Create a new dataframe to use while maintaining the original
new_df = df.copy()

# Extract year from the new dataframe
new_df['year'] = new_df['Review Date'].dt.year

# Group by relevant columns
review_date = new_df.groupby('year')['Review'].count()
review_date = pd.DataFrame(review_date).reset_index()

# plot the data
review_date.plot.line(x = 'year', y = 'Review', color = 'green', figsize=(15, 8));


The rate of customer reviews took a major increase from 2020. This could be attributed to the Covid Pandemic where people spent lots of time indoors and online shopping was at an all time high.

### 6.6. Relationship between price and Ratings

In [39]:
# Group by relevant columns
review_price = new_df.groupby('Price')['Rating'].count()
review_price = pd.DataFrame(review_price).reset_index()

# Plot the data
fig, ax = plt.subplots(figsize=(15,8))
sns.scatterplot(x = 'Price', y = 'Rating', data = review_price, color = 'green')
plt.show();

The scatter plot above does not show a correlation between price and rating.

# 7. Implementing the Solution

## 7.1 Preprocessing

- tokenization
- lowercasing our words
- lemmatization/stemming
- vectorization

The first step will be to perform a contraction the reviews, to make sure that our words are expanded, for example, `isn't` would be expanded to `is not`

In [40]:
# fixing contractions
def text_contraction(text):
  
  # creating an empty list
  expanded_words = []

  for word in text.split():
    # using contractions.fix to expand the shortened words
    expanded_words.append(contractions.fix(word))  
    
  expanded_text = ' '.join(expanded_words)

  return expanded_text

In [41]:
# apply the contraction funtion to our reviews

df['Review Title'] = df['Review Title'].map(lambda x: text_contraction(x))
df['Review'] = df['Review'].map(lambda x: text_contraction(x))
df.head()

,Model_Type,Brand,Price,Review Date,Rating,Review Title,Review
0,RCA Reno,RCA,69,2022-02-11,4,".. not what ordered, not New... but it works s...","So first off...it is not what I ordered, but I..."
2,RCA Reno,RCA,69,2022-06-07,3,Will not work on T-Mobile sysem!,New phone write up indicates T-Mobile system c...
3,RCA Reno,RCA,69,2022-04-14,3,A burner or for a kid,I use this as a burner w/o a sim card in it. J...
4,RCA Reno,RCA,69,2022-08-13,4,It works okay,It works fine
5,RCA Reno,RCA,69,2022-05-10,3,Phone,"So far I do not like this phone at all, I thre..."


In [42]:
sentence = 'Today is Monday, 3 pm'

punctuation = string.punctuation

  # lower case our string
  # text = str([word.lower() for word in t
  # remove the digits
sentence = re.sub('\d', '', sentence)

  # cretae our word tokens
tokens = word_tokenize(sentence)

print(tokens)

['Today', 'is', 'Monday', ',', 'pm']


In [43]:
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [44]:
from nltk.corpus import stopwords

In [45]:
# grab the stop words from stopwords.words('english)
#stopwords_list = stopwords.words('english')
#stopwords_list += punctuation

stopwords_list = stopwords.words('english') + list(string.punctuation)
stopwords_list += ["''", '""', '...', '``']

# remove the stop words
clean_list = [word.lower() for word in tokens if word.lower() not in stopwords_list]

print(clean_list)

['today', 'monday', 'pm']


In [46]:
# tokenization

def tokenize_words(text):
  # grab all the punctuations
  punctuation = string.punctuation

  # lower case our string
  # text = str([word.lower() for word in t
  # remove the digits
  text = re.sub('\d', '', text)

  # cretae our word tokens
  tokens = word_tokenize(text)

  # grab the stop words from stopwords.words('english)
  # stopwords_list = stopwords.words('english')
  # stopwords_list += punctuation
  # stopwords_list = stopwords.words('english') + list(string.punctuation)
  # stopwords_list += ["''", '""', '...', '``', '..', '....']
  punctuation_list = list(string.punctuation)
  punctuation_list += ["''", '""', '...', '``', '..', '....']

  # remove the stop words
  clean_list = [word.lower() for word in tokens if word.lower() not in punctuation_list]
  
  # return a clean tokenized set
  return clean_list

In [47]:
df['Review'] = df['Review'].map(lambda x: tokenize_words(x))
df['Review Title'] = df['Review Title'].map(lambda x: tokenize_words(x))
df.head()

,Model_Type,Brand,Price,Review Date,Rating,Review Title,Review
0,RCA Reno,RCA,69,2022-02-11,4,"[not, what, ordered, not, new, but, it, works,...","[so, first, off, it, is, not, what, i, ordered..."
2,RCA Reno,RCA,69,2022-06-07,3,"[will, not, work, on, t-mobile, sysem]","[new, phone, write, up, indicates, t-mobile, s..."
3,RCA Reno,RCA,69,2022-04-14,3,"[a, burner, or, for, a, kid]","[i, use, this, as, a, burner, w/o, a, sim, car..."
4,RCA Reno,RCA,69,2022-08-13,4,"[it, works, okay]","[it, works, fine]"
5,RCA Reno,RCA,69,2022-05-10,3,[phone],"[so, far, i, do, not, like, this, phone, at, a..."


After tokenization, lemmatization is done to decompose the words to their most basic forms(lemma), but before that, tagging the words is done to ensure that the lemmatization gets the parts of speech represented by the words correctly.

In [48]:
# create a function that takes in the nltk POS tags
# and transforms them to wordnet tags
def wordnet_pos(word_tag):
    '''
    Translate nltk POS to wordnet tags
    '''
    if word_tag.startswith('J'):
        return wordnet.ADJ
    elif word_tag.startswith('V'):
        return wordnet.VERB
    elif word_tag.startswith('N'):
        return wordnet.NOUN
    elif word_tag.startswith('R'):
        return wordnet.ADV
    else:
        return wordnet.NOUN

In [49]:
# check to see how the word tag performs
sentence = df["Review"][0]
pos_tag(sentence[0:20])

[('so', 'RB'),
 ('first', 'RB'),
 ('off', 'IN'),
 ('it', 'PRP'),
 ('is', 'VBZ'),
 ('not', 'RB'),
 ('what', 'WP'),
 ('i', 'VB'),
 ('ordered', 'VBD'),
 ('but', 'CC'),
 ('i', 'NNS'),
 ('do', 'VBP'),
 ('not', 'RB'),
 ('really', 'RB'),
 ('care', 'VB'),
 ('as', 'IN'),
 ('it', 'PRP'),
 ('is', 'VBZ'),
 ('a', 'DT'),
 ('burner.i', 'NN')]

In [50]:
# inastantiate the lemmatizer
lemmatizer = WordNetLemmatizer() 

In [51]:
df['Review'][0:2]

0    [so, first, off, it, is, not, what, i, ordered...
2    [new, phone, write, up, indicates, t-mobile, s...
Name: Review, dtype: object

In [52]:
def word_lemma(text):
    '''
    Translate the text POS tags to Word net tags then pass it to the lemmatizer
    
    '''
    # get the pos tags for the text
    word_pos_tags = pos_tag(text)
    
    # translate the pos tags to word net tags
    word_net_tag = [(text[0], wordnet_pos(text[1])) for text in word_pos_tags]
    
    # Pass the text with the wordnet tags to the lemmatizer
    lemma_word = [lemmatizer.lemmatize(text[0], text[1]) for text in word_net_tag]
    
    return lemma_word
    

In [53]:
nltk.download('all', quiet=True)
df['Review'] = df['Review'].apply(word_lemma)
df['Review Title'] = df['Review Title'].apply(word_lemma)
df.head()

,Model_Type,Brand,Price,Review Date,Rating,Review Title,Review
0,RCA Reno,RCA,69,2022-02-11,4,"[not, what, order, not, new, but, it, work, so...","[so, first, off, it, be, not, what, i, order, ..."
2,RCA Reno,RCA,69,2022-06-07,3,"[will, not, work, on, t-mobile, sysem]","[new, phone, write, up, indicate, t-mobile, sy..."
3,RCA Reno,RCA,69,2022-04-14,3,"[a, burner, or, for, a, kid]","[i, use, this, a, a, burner, w/o, a, sim, card..."
4,RCA Reno,RCA,69,2022-08-13,4,"[it, work, okay]","[it, work, fine]"
5,RCA Reno,RCA,69,2022-05-10,3,[phone],"[so, far, i, do, not, like, this, phone, at, a..."


Frequency Distribution Plot

In [54]:
# create the frequency distribution plot
sample = df['Review']
freq1_dist = []

for review in sample:
    freq1_dist.extend(review)
    
fdist = FreqDist(freq1_dist)
plt.figure(figsize=(15, 10))
fdist.plot(50);

In [55]:
# display the count of the first 200 texts
word_frequency = fdist.most_common(200)
word_frequency[:30]

[('be', 23219),
 ('the', 21331),
 ('i', 15870),
 ('a', 14541),
 ('it', 14475),
 ('phone', 14278),
 ('and', 12184),
 ('to', 11830),
 ('not', 9283),
 ('have', 7822),
 ('this', 7262),
 ('for', 6461),
 ('of', 5009),
 ('do', 4970),
 ('with', 4924),
 ('my', 4841),
 ('that', 4320),
 ('but', 4168),
 ('in', 4010),
 ('on', 3971),
 ('work', 3442),
 ('you', 3365),
 ('use', 2919),
 ('good', 2854),
 ('get', 2760),
 ('so', 2609),
 ('great', 2349),
 ('battery', 2246),
 ('like', 2005),
 ('can', 1962)]

In [56]:
# convert the list into a string

X = df['Review']

df["Review"]= X.map(lambda x: ' '.join(map(str, x)))
df["Review"]

0        so first off it be not what i order but i do n...
2        new phone write up indicate t-mobile system co...
3        i use this a a burner w/o a sim card in it jus...
4                                             it work fine
5        so far i do not like this phone at all i throw...
                               ...                        
17162    what be not to like work fantastic on t-mobile...
17163    i like this phone it work very well it do not ...
17164    i be not super tech savvy i just want a reliab...
17165                                            good deal
17167    whoa my first blu phone and i be never buy ano...
Name: Review, Length: 9144, dtype: object

Vader Analysis

In [57]:
df["id"] = df.index + 1
df = df.reindex(columns=['id', 'Model_Type', 'Brand', 'Price', 'Review Date', 
                         'Rating', 'Review Title', 'Review'])
df.drop("Review Date", axis=1, inplace=True)
df.head()

,id,Model_Type,Brand,Price,Rating,Review Title,Review
0,1,RCA Reno,RCA,69,4,"[not, what, order, not, new, but, it, work, so...",so first off it be not what i order but i do n...
2,3,RCA Reno,RCA,69,3,"[will, not, work, on, t-mobile, sysem]",new phone write up indicate t-mobile system co...
3,4,RCA Reno,RCA,69,3,"[a, burner, or, for, a, kid]",i use this a a burner w/o a sim card in it jus...
4,5,RCA Reno,RCA,69,4,"[it, work, okay]",it work fine
5,6,RCA Reno,RCA,69,3,[phone],so far i do not like this phone at all i throw...


In [58]:
# creating polarity scores on the entire dataset
from nltk.sentiment import SentimentIntensityAnalyzer
nltk.download('vader_lexicon')
sia = SentimentIntensityAnalyzer()
from tqdm.notebook import tqdm
res = {}
for i, row in tqdm(df.iterrows(), total=len(df)):
  text = row["Review"]
  id = row["id"]
  res[id] = sia.polarity_scores(text)


[nltk_data] Downloading package vader_lexicon to /root/nltk_data...
[nltk_data]   Package vader_lexicon is already up-to-date!


  0%|          | 0/9144 [00:00<?, ?it/s]

In [59]:
# populating results into a df
vader = pd.DataFrame(res).T
vader = vader.reset_index().rename(columns={"index":"id"})
vader_df = vader.merge(df, how="left")

In [60]:
# sentiment scores combined with the meta data


# pd.set_option('display.max_colwidth',10000)
vader_df.head()

,id,neg,neu,pos,compound,Model_Type,Brand,Price,Rating,Review Title,Review
0,1,0.103,0.806,0.091,-0.6901,RCA Reno,RCA,69,4,"[not, what, order, not, new, but, it, work, so...",so first off it be not what i order but i do n...
1,3,0.058,0.921,0.021,-0.6480,RCA Reno,RCA,69,3,"[will, not, work, on, t-mobile, sysem]",new phone write up indicate t-mobile system co...
2,4,0.055,0.748,0.197,0.9585,RCA Reno,RCA,69,3,"[a, burner, or, for, a, kid]",i use this a a burner w/o a sim card in it jus...
3,5,0.000,0.526,0.474,0.2023,RCA Reno,RCA,69,4,"[it, work, okay]",it work fine
4,6,0.247,0.753,0.000,-0.8022,RCA Reno,RCA,69,3,[phone],so far i do not like this phone at all i throw...


In [61]:
# plotting a bar plot to compare rating and compound value to see if the ratings align with the sentiment scores
df_sorted = vader_df.sort_values('Rating')
fig, ax = plt.subplots(figsize=(10, 8))
sns.barplot(data=df_sorted, x = "Rating", y="compound")
ax.set_title("Compound Score by Amazon Star Rating")
plt.show()

Making assumptions on our data, if rating is 5, then it is likely to be a more positive value than a rating of 1.

In [62]:
fig, axs = plt.subplots(1, 3, figsize=(15, 5))

sns.barplot(data=df_sorted, x = "Rating", y="pos", ax=axs[0])
sns.barplot(data=df_sorted, x = "Rating", y="neu", ax=axs[1])
sns.barplot(data=df_sorted, x = "Rating", y="neg", ax=axs[2])
axs[0].set_title("Positive")
axs[1].set_title("Neutral")
axs[2].set_title("Negative")
plt.tight_layout()
plt.show()

The more positive the compound value, the higher the Ranking score. Positive values represent high rankings of 4 and 5 while low rankings of 1 and 2 mainly show negative values. This proves the assumption made on the data.

## Modeling

In [63]:
# splitting data 

df_train, df_test = train_test_split(
  df,
  test_size=0.3,
  random_state=23
)
df_val, df_test = train_test_split(
  df_test,
  test_size=0.5,
  random_state=23
)

### Vader Analysis

In [64]:
# Exploring vader as a model for sentiment analysis

analyzer = SentimentIntensityAnalyzer()

In [65]:
df_train["scores"] = df_train["Review"].apply(lambda review : analyzer.polarity_scores(review))
df_train["compound"] = df_train["scores"].apply(lambda score_dict :score_dict["compound"])
df_train["train_label"] = df_train["compound"].apply(lambda c : "positive" if c >= 0.05 else "negative" if c <= -0.05 else "neutral")

df_train.head()

,id,Model_Type,Brand,Price,Rating,Review Title,Review,scores,compound,train_label
9655,9656,OnePlus Nord N10,OnePlus,239,2,"[not, ready, for, prime, time]",primary gripe be the screen brightness -- or l...,"{'neg': 0.143, 'neu': 0.784, 'pos': 0.073, 'co...",-0.3612,negative
840,841,BLU Tank II,BLU,26,5,"[worth, the, price]",i have a very clear requirement for a phone wh...,"{'neg': 0.0, 'neu': 0.734, 'pos': 0.266, 'comp...",0.8968,positive
13564,13565,Samsung Galaxy S20+,Amazon Refurbished,340,5,"[work, great]",easy to use,"{'neg': 0.0, 'neu': 0.408, 'pos': 0.592, 'comp...",0.4404,positive
9628,9629,OnePlus Nord N10,OnePlus,239,1,[battery],after a few month of use when i sometimes turn...,"{'neg': 0.028, 'neu': 0.909, 'pos': 0.063, 'co...",0.4215,positive
14023,14024,Apple iPhone 8,Amazon Refurbished,159,5,"[great, price]",iphone be a gift i be not sure what to expect ...,"{'neg': 0.038, 'neu': 0.533, 'pos': 0.43, 'com...",0.9550,positive


In [66]:
# Exploring the unique elements in label column

df_train["train_label"].value_counts()

positive    4004
negative    1565
neutral      831
Name: train_label, dtype: int64

In [67]:
# plotting distribution of sentiment scores
colors = ['green', 'blue', 'magenta']
df_train["train_label"].value_counts().plot(kind="bar", color = colors, figsize=(10, 8))
plt.xlabel("Sentiment Label", size=12)
plt.ylabel("Count", size=12)
plt.title("Distribution of Sentiment Scores", size=14)

Text(0.5, 1.0, 'Distribution of Sentiment Scores')

In [68]:
# plotting a pie chart
colors = ['green', 'blue', 'magenta']
df_train["train_label"].value_counts().plot(figsize = (8, 8), kind='pie', y='Sentiment Analysis Results for Amazon Mobile Reviews', colors= colors, autopct='%1.0f%%')
plt.legend(df_train.train_label)
plt.title("Distribution of Sentiment Scores", size=14)
plt.show()

In [69]:
# Evaluating the vader analysis

df_test["scores"] = df_test["Review"].apply(lambda review : analyzer.polarity_scores(review))
df_test["compound_test"] = df_test["scores"].apply(lambda score_dict :score_dict["compound"])
df_test["test_label"] = df_test["compound_test"].apply(lambda c : "positive" if c >= 0.05 else "negative" if c <= -0.05 else "neutral")


In [70]:
df_test.head()

,id,Model_Type,Brand,Price,Rating,Review Title,Review,scores,compound_test,test_label
17020,17021,BLU F91 5G,BLU,229,4,"[great, value]",this phone work a expect it be definitely a bu...,"{'neg': 0.0, 'neu': 0.769, 'pos': 0.231, 'comp...",0.5423,positive
5978,5979,Samsung Galaxy S20,Amazon Refurbished,314,5,"[good, buy, beautiful, phone, like, new]",love the phone,"{'neg': 0.0, 'neu': 0.323, 'pos': 0.677, 'comp...",0.6369,positive
6450,6451,Samsung Galaxy S20,Amazon Refurbished,314,5,"[great, device]",wish it have a little more customizability on ...,"{'neg': 0.0, 'neu': 0.886, 'pos': 0.114, 'comp...",0.4019,positive
3893,3894,Samsung Galaxy A10e,Amazon Refurbished,102,5,[bien],nitido,"{'neg': 0.0, 'neu': 1.0, 'pos': 0.0, 'compound...",0.0000,neutral
2857,2858,Moto G Power,Motorola,149,5,"[greatnphone, fast, delivery]",fast delivery great phone,"{'neg': 0.0, 'neu': 0.423, 'pos': 0.577, 'comp...",0.6249,positive


### Text Blob

In [71]:
# import the model
from textblob import TextBlob

In [72]:
# Implementing the textblob analysis

df[["polarity", "subjectivity"]] = df["Review"].apply(lambda Text: pd.Series(TextBlob(Text).sentiment))

In [73]:
# exploring the positive comments

df[df.polarity > 0].tail()

,id,Model_Type,Brand,Price,Rating,Review Title,Review,polarity,subjectivity
17162,17163,BLU F91 5G,BLU,229,2,"[what, a, great, phone]",what be not to like work fantastic on t-mobile...,0.408712,0.471402
17163,17164,BLU F91 5G,BLU,229,5,"[great, android, phone, with, long, battery, l...",i like this phone it work very well it do not ...,0.370000,0.413704
17164,17165,BLU F91 5G,BLU,229,5,"[great, phone]",i be not super tech savvy i just want a reliab...,0.262195,0.568817
17165,17166,BLU F91 5G,BLU,229,5,"[good, deal]",good deal,0.700000,0.600000
17167,17168,BLU F91 5G,BLU,229,5,"[eat, your, heart, out, apple, samsung]",whoa my first blu phone and i be never buy ano...,0.286111,0.455000


In [74]:
# exploring the negative comments

df[df.polarity < 0].head()

,id,Model_Type,Brand,Price,Rating,Review Title,Review,polarity,subjectivity
2,3,RCA Reno,RCA,69,3,"[will, not, work, on, t-mobile, sysem]",new phone write up indicate t-mobile system co...,-0.070455,0.277652
10,11,RCA Reno,RCA,69,3,"[broke, in, less, than, a, month]",i be sorry to have to do this but i have to ch...,-0.278175,0.550794
18,19,RCA Reno,RCA,69,2,"[horrible, battery, life, and, i, mean, horrible]",this phone work for basic function but be extr...,-0.150000,0.262500
19,20,RCA Reno,RCA,69,4,"[do, what, i, expect, it, to]",take my hello mobile sim card with no problem ...,-0.075000,0.700000
29,30,RCA Reno,RCA,69,1,"[do, not, waste, your, money]",these phone be so slow outdated and problemati...,-0.117857,0.429167


In [75]:
# exploring neutral comments

df[df.polarity == 0].head()

,id,Model_Type,Brand,Price,Rating,Review Title,Review,polarity,subjectivity
11,12,RCA Reno,RCA,69,4,"[seam, ok]",just get it go let u see,0.0,0.000000
64,65,OnePlus Nord N20,OnePlus,299,5,[good],thank you,0.0,0.000000
91,92,Jteman 5in1 Bluetooth,JTEMAN,16,3,"[do, not, get, my, phone, to, sound, loud]",could not plug headphone in the product,0.0,0.000000
114,115,Jteman 5in1 Bluetooth,JTEMAN,16,3,"[be, careful]",not as substantial a i want,0.0,0.000000
124,125,Jteman 5in1 Bluetooth,JTEMAN,16,4,"[easy, to, use]",it be very simple to setup,0.0,0.464286


## Roberta Model: Neural Network Model

In [ ]:
# Transformer models account for context
# !pip install transformers
# from transformers import AutoTokenizer
# from transformers import AutoModelForSequenceClassification
# from scipy.special import softmax

In [ ]:
# # transfer learning on a pretrained model
# # finding pretrained weights to use in the model
# MODEL = f"cardiffnlp/twitter-roberta-base-sentiment"
# tokenizer = AutoTokenizer.from_pretrained(MODEL)
# model = AutoModelForSequenceClassification.from_pretrained(MODEL)

In [ ]:
# def polarity_scores_roberta(words):
#   encoded_text = tokenizer(words, return_tensors="pt")
#   output = model(**encoded_text)
#   scores = output [0][0].detach().numpy()
#   scores = softmax(scores)
#   scores_dict ={
#       "roberta_neg": scores[0],
#       "roberta_neu" : scores[1],
#       "roberta_pos" : scores [2]
#   }
#   return scores_dict


In [ ]:
# res = {}
# for i, row in tqdm(df.iterrows(), total=len(df)):
#   try:
#     text = row["Review"]
#     id = row["id"]
#     roberta_results = polarity_scores_roberta(text)
#   except RuntimeError:
#     print(f'Broke for id {id}')

In [ ]:
# vader = pd.DataFrame(res).T
# vader = vader.reset_index().rename(columns={"index":"id"})
# vader_df = vader.merge(df, how="left")

### Bert Model

In [ ]:
! pip install torch torchvision torchaudio --extra-index-url https://download.pytorch.org/whl/cu116

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/, https://download.pytorch.org/whl/cu116


In [ ]:
! pip install transformers

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 4.7 MB 7.3 MB/s 
     |████████████████████████████████| 6.6 MB 46.9 MB/s 
     |████████████████████████████████| 120 kB 52.4 MB/s 


In [ ]:
from transformers import AutoTokenizer, AutoModelForSequenceClassification


In [ ]:
import torch

# 8. Challenging the Solution

# 9. Conclusions

# 10. Recommendations